## Data exploration and feature selection

In this notebook I will explore the credit card transaction data downloaded from https://www.kaggle.com/mlg-ulb/creditcardfraud/data and use various techniques to select features for modeling. Feature selection reduces noise and allows the model to train on the most important information. Often time this helps improve the training speed as well as model performance.

In [1]:
import pandas as pd
import numpy as np 

import tensorflow as tf

from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
df = pd.read_csv("creditcard.csv")

In [3]:
if ~any(df.isnull().sum()):
    print('No missing values.')

No missing values.


In [4]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


A few things to notice:
1. There are 30 features: Time, Amount, and 28 others that are masked.
2. The data is highly imbalanced: from the mean value of Class we see that only 0.17% of the transactions are fraud.
3. The variances for V1 to V28 are already sorted. This suggests that V1-V28 have been already been PCA transformed.

In [5]:
array = df.values
X = array[:,1:29]
pca = PCA()
X_new = pca.fit_transform(X)

In [6]:
abs(X_new)-abs(X)

array([[-1.72528658e-13,  7.87286902e-14, -2.88213897e-13, ...,
         1.11577414e-14,  8.29891711e-15,  8.00817745e-14],
       [-8.88178420e-16, -8.74855743e-14,  1.28147493e-13, ...,
        -2.17326157e-14, -4.03271166e-14, -4.84855212e-15],
       [ 1.33226763e-15, -8.88178420e-15,  7.79376563e-14, ...,
         3.72479825e-14,  2.97747937e-14,  2.44179676e-14],
       ...,
       [ 0.00000000e+00,  1.12132525e-14, -7.99360578e-15, ...,
         3.81639165e-15,  3.92047506e-16, -1.57859836e-15],
       [ 1.41553436e-15,  3.66373598e-15, -4.32986980e-15, ...,
         1.33226763e-15,  3.09474668e-15,  3.53883589e-15],
       [ 1.22124533e-15, -8.32667268e-16,  3.21964677e-15, ...,
        -1.44328993e-15,  3.73832909e-16, -4.57966998e-16]])

Applying Principal Component Analysis doesn't change the features except for some sign flips (due to the ambiguity in choosing a sign for the eigenvectors), so this does confirm that the given data has been PCA transformed.

In [7]:
n_PCA = 28 # number of features to keep after PCA
features_PCA = ['V' + str(i) for i in range(1,n_PCA+1)]
print(features_PCA)

['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']


Next I will do supervised feature selection by choosing features that have the strongest correlation with the class. This can be done with the built-in SelectKBest function from scikit-learn. Here I use f_classif as score function (chi squared is for non-negative data). Optional: use ExtraTreesClassifier to fit the model to extract the importance of features.

In [8]:
X = array[:,1:29]
y = array[:,30]
select = SelectKBest(score_func=f_classif, k = 'all')
fit = select.fit(X, y)
scores_SKB = pd.DataFrame(fit.scores_)

n_selectK = 15 # number of features to keep after SelectK
features_selectK = ['V'+ str(i+1) for i in scores_SKB.nlargest(n_selectK,0).index]
print(features_selectK)

#model = ExtraTreesClassifier()
#model.fit(X,y)
#scores_ET = pd.DataFrame(model.feature_importances_)

#n_ET = 15 # number of important features to keep after ExtraTrees
#features_ET = ['v'+ str(i+1) for i in scores_ET.nlargest(n_ET,0).index]
#print(features_ET)

C:\Anaconda\envs\python36\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


['V17', 'V14', 'V12', 'V10', 'V16', 'V3', 'V7', 'V11', 'V4', 'V18', 'V1', 'V9', 'V5', 'V2', 'V6']


In [9]:
kept_features = list(set(features_selectK).intersection(features_PCA))
print(kept_features)

['V11', 'V2', 'V7', 'V12', 'V4', 'V14', 'V1', 'V3', 'V5', 'V6', 'V9', 'V10', 'V17', 'V16', 'V18']


In [10]:
dropped_features = ['V'+str(i) for i in range(1,29) if ('V'+str(i)) not in kept_features]
#print(dropped_features)
df = df.drop(dropped_features, axis=1)

In [11]:
# prepare training and testing data

df.rename(columns={'Class':'Fraud'}, inplace=True)
df['NonFraud'] = 1-df['Fraud']

X = df.drop(['Fraud','NonFraud'],1)
y = df[['Fraud','NonFraud']]

# normalize the data to help the optimization
for col in X.columns:
    temp = X.loc[:,col]
    mean, std = temp.mean(), temp.std()
    X.loc[:,col] = (temp - mean) / std

# shuffle and split data into training and testing sets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)
for idx_train, idx_test in sss.split(X, y):
    X_train = X.iloc[idx_train]
    y_train = y.iloc[idx_train]
    X_test = X.iloc[idx_test]
    y_test = y.iloc[idx_test]

## Train and test the neural network

In [16]:
n_epochs = 200 # number of training epochs
steps_to_check = 5

batch_size = 1000
learning_rate = 0.01
keep_prob = 0.9 # for dropout method, to reduce overfitting

n_nodes0 = len(kept_features) + 2 # number of input nodes
n_nodes1 = 20 # number of nodes in the 1st hidden layer
n_nodes2 = 25 # number of nodes in the 2nd hidden layer

# set up the variables
pkeep = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32, [None, n_nodes0])

W1 = tf.Variable(tf.random_normal([n_nodes0, n_nodes1], stddev=0.1))
b1 = tf.Variable(tf.zeros([n_nodes1]))
y1 = tf.nn.sigmoid(tf.matmul(x, W1) + b1)

W2 = tf.Variable(tf.random_normal([n_nodes1, n_nodes2], stddev=0.1))
b2 = tf.Variable(tf.zeros([n_nodes2]))
y2 = tf.nn.sigmoid(tf.matmul(y1, W2) + b2)
y2 = tf.nn.dropout(y2, pkeep)

W3 = tf.Variable(tf.random_normal([n_nodes2, 2], stddev=0.1)) 
b3 = tf.Variable(tf.zeros([2]))
y = tf.nn.softmax(tf.matmul(y2, W3) + b3)
y_ = tf.placeholder(tf.float32, [None, 2])

# use cross entropy as cost function
cost_function = -tf.reduce_sum(y_*tf.log(y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost_function)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    # initialize the variables
    sess.run(tf.global_variables_initializer())
    
    # train the network
    for epoch in range(n_epochs): 
        for batch in range(int(len(X_train)/batch_size)):
            X_batch = X_train[batch*batch_size:(1+batch)*batch_size]
            y_batch = y_train[batch*batch_size:(1+batch)*batch_size]
            sess.run([optimizer], {x: X_batch, y_: y_batch, pkeep: keep_prob})

        # check cost and accuracy after fixed number of steps
        if epoch % steps_to_check == 0:
            acc, cost = sess.run([accuracy, cost_function], 
                                 {x: X_train, y_: y_train, pkeep: keep_prob})
            print('Epoch: %4d    Cost: %8.2f    Accuracy: %.5f' %(epoch, cost, acc))
    
    print('\nTraining completed.\n')
    print('Testing the model on test data.\n')
    
    # test on test data
    output = sess.run(y, {x: X_test, y_: y_test, pkeep:1})

In [ ]:
target = (y_test['Fraud'] > 0).astype(int)
predict = (output[:,0] > output[:,1]).astype(int)
from evaluation import confusion
confusion(predict,target)

There are a lot of parameters to tune in order to optimize the performance of the model (size of the neural networks, number of training epochs, number of important features to keep etc.). Generally the accuracy is above 99.92%. The recall and precision for the positive class (fraud) can be around 80%, and those for the negative class (non-fraud) is above 99.96%.